для домашнего задания взяла датасет train 
https://www.kaggle.com/c/titanic/data?select=train.csv

In [268]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [326]:
data = pd.read_csv("titanic_train.csv")
data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [327]:
data.shape

(891, 12)

In [328]:
categorical_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
continuous_columns = ['Age', 'Fare']
to_drop = ['PassengerId', 'Name', 'Cabin', 'Ticket', 'Survived']

In [329]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [330]:
len(data) - data.count()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [331]:
mean = data['Age'].mean()
data['Age']=data['Age'].fillna(mean)
data['Embarked']=data['Embarked'].fillna('S')

У нас есть 7 признаков и 1 целевая переменная (бинарная) - нужно определить выжил человек или нет
Всего 891 человек

Посмотрим на соотношение классов

In [332]:
data['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [333]:
#categorical_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']

In [334]:
data= pd.get_dummies(data, prefix='Pclass_', columns=['Pclass'])
data= pd.get_dummies(data, prefix='SibSp', columns=['SibSp'])
data= pd.get_dummies(data, prefix='Parch', columns=['Parch'])
data= pd.get_dummies(data, prefix='Embarked', columns=['Embarked'])
data.loc[data['Sex']=='male', 'Sex'] = 1
data.loc[data['Sex']=='female', 'Sex'] = 0
data['Sex']=data['Sex'].astype(int)

In [335]:
y_data = data['Survived'].copy()
data.drop(columns = to_drop, axis = 0, inplace = True )
x_data = data.copy()

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [336]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [337]:
model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверяем качество

In [338]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return f1, roc, rec, prc

    
f1, roc, rec, prc = evaluate_results(y_test, y_predict)

Classification results:
f1: 71.88%
roc: 77.42%
recall: 66.67%
precision: 77.97%


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [339]:
data['Servived'] = y_data

In [340]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 86/342 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [341]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    805
 1     86
Name: class_test, dtype: int64


In [342]:
mod_data.head(10)

,Sex,Age,Fare,Pclass__1,Pclass__2,Pclass__3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,...,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S,Servived,class_test
0,1,22.000000,7.2500,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,-1
1,0,38.000000,71.2833,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,-1
2,0,26.000000,7.9250,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,1,-1
3,0,35.000000,53.1000,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,-1
4,1,35.000000,8.0500,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,-1
5,1,29.699118,8.4583,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-1
6,1,54.000000,51.8625,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,-1
7,1,2.000000,21.0750,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,-1
8,0,27.000000,11.1333,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,1,1,-1
9,0,14.000000,30.0708,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [343]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling

In [344]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(86, 25) (86, 25)


In [345]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
f12, roc2, rec2, prc2 = evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 67.90%
roc: 76.66%
recall: 85.96%
precision: 56.11%


### 2. probabilistic approach

* The training set will be divided into a fitting-set that will be used to fit the estimator in order to estimate P(s=1|X) and a held-out set of positive samples that will be used to estimate P(s=1|y=1)¶

In [346]:
def fit_PU_estimator(X,y, hold_out_ratio, estimator):
    
    # find the indices of the positive/labeled elements
    assert (type(y) == np.ndarray), "Must pass np.ndarray rather than list as y"
    positives = np.where(y == 1.)[0] 
    # hold_out_size = the *number* of positives/labeled samples 
    # that we will use later to estimate P(s=1|y=1)
    hold_out_size = int(np.ceil(len(positives) * hold_out_ratio))
    np.random.shuffle(positives)
    # hold_out = the *indices* of the positive elements 
    # that we will later use  to estimate P(s=1|y=1)
    hold_out = positives[:hold_out_size] 
    # the actual positive *elements* that we will keep aside
    X_hold_out = X[hold_out] 
    # remove the held out elements from X and y
    X = np.delete(X, hold_out,0) 
    y = np.delete(y, hold_out)
    # We fit the estimator on the unlabeled samples + (part of the) positive and labeled ones.
    # In order to estimate P(s=1|X) or  what is the probablity that an element is *labeled*
    estimator.fit(X, y)
    # We then use the estimator for prediction of the positive held-out set 
    # in order to estimate P(s=1|y=1)
    hold_out_predictions = estimator.predict_proba(X_hold_out)
    #take the probability that it is 1
    hold_out_predictions = hold_out_predictions[:,1]
    # save the mean probability 
    c = np.mean(hold_out_predictions)
    return estimator, c

def predict_PU_prob(X, estimator, prob_s1y1):
    predicted_s = estimator.predict_proba(X)
    predicted_s = predicted_s[:,1]
    return predicted_s / prob_s1y1

* test the PU estimation approach

In [347]:
predicted = np.zeros(len(x_data))
learning_iterations = 24
for index in range(learning_iterations):
    pu_estimator, probs1y1 = fit_PU_estimator(x_data, y_labeled, 0.2, xgb.XGBClassifier())
    predicted += predict_PU_prob(x_data, pu_estimator, probs1y1)
    if(index%4 == 0): 
        print(f'Learning Iteration::{index}/{learning_iterations} => P(s=1|y=1)={round(probs1y1,2)}')

Learning Iteration::0/24 => P(s=1|y=1)=0.1599999964237213
Learning Iteration::4/24 => P(s=1|y=1)=0.2199999988079071
Learning Iteration::8/24 => P(s=1|y=1)=0.2199999988079071
Learning Iteration::12/24 => P(s=1|y=1)=0.17000000178813934
Learning Iteration::16/24 => P(s=1|y=1)=0.20999999344348907
Learning Iteration::20/24 => P(s=1|y=1)=0.07000000029802322


* compare the performance of the predictions of the PU approacj (y_predict) with the actuall original classes (y_positive) that we have saved aside¶

In [348]:
y_predict = [1 if x > 0.01 else 0 for x in (predicted/learning_iterations)]
f13, roc3, rec3, prc3 = evaluate_results(y_positive, y_predict)

Classification results:
f1: 62.52%
roc: 64.01%
recall: 92.69%
precision: 47.17%


In [349]:
result = {"F-Score":np.array([f1, f12, f13]),
          "roc":np.array([roc, roc2, roc3]),
          "Recall":np.array([rec, rec2, rec3]),
        "precision": np.array([prc, prc2, prc3])}

results = pd.DataFrame(result, index=['classication', 'NS', 'PA'])
print(results)

               F-Score       roc    Recall  precision
classication  0.718750  0.774242  0.666667   0.779661
NS            0.678992  0.766564  0.859574   0.561111
PA            0.625247  0.640135  0.926901   0.471726


### Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

Ваш ответ здесь: